In [1]:
# Ion Chemistry

"""
Preparation (could be written as module)
- load file with reaction and species information, inkl. stochiometry and charges etc.
- produce classes with sepcies concentration etc
- produce classes with reaction information
"""

import numpy as np
from scipy.interpolate import CubicSpline, interp1d
import import_ipynb
from organizing import pcfg
con = pcfg

class constituent():
    """
    For every constituent of the ionosphere, this class is generated to record its
    -density
    -production
    -loss
    and from that calculate the density of the new timestep.
    All these values are in the form of an array as a height profile.    
    """
    def __init__(self, c_ID, name, charge, density):
        self.c_ID = int(c_ID)
        self.name = name
        self.density = density
        self.charge = int(charge)
        self.loss = np.zeros(len(self.density))
        self.prod = np.zeros(len(self.density))
        self.dndt = np.zeros(len(self.density))
    
    def log(self, density):
        raise NotImplementedError
        
    def integrate(self, prod, loss):
        self.dndt = prod - loss
        return NotImplementedError
    
    def iterate_time(self, dt):
        self.dndt = self.prod - self.loss
        self.density = self.density + self.dndt * dt
        self.density[self.density < 0] = 0
        self.prod = np.zeros(len(self.density))
        self.loss = np.zeros(len(self.density))
        #self.log(density)

        

class reaction():
    """
    Defines all parameters of a chemical reaction in the ionosphere
    
    Parameters:
    r_ID: Name of the reaction (according to Brekke of Schunk and Nagy)
        string
    r_stoch: Reaction in chemical symbols, with stochiometry
        string
    educts: constituents that go into the reaction (lefthand side of r_stoch)
        list of strings
    products: constituents that come out of the reaction (righthand side of r_stoch)
        list of strings
    r_rate_string: the reaction rate, ready to be interpreted as a equation in python (with variables)
        string
        
    Methods:
    r_rate: evaluates the reaction rate at a given temperature T
        float
    """    
    def __init__(self, ionChem_inst, r_ID, r_stoch, educts, products, r_rate_string, all_species):
        self.r_ID = r_ID
        self.r_stoch = r_stoch
        self.educts = educts
        self.educts_ID = ionChem_inst.getConstituentsIDByName(self.educts)
        self.products = products
        self.products_ID = ionChem_inst.getConstituentsIDByName(self.products)
        self.r_rate_string = r_rate_string
        
    def r_rate(self, T):
        Te = T
        Tr = T
        self.rr = eval(self.r_rate_string)
        return self.rr
    
    


class ionChem:
    """
    Defines the ion chemistry model.
    """
    
    def __init__(self, reactions_file, z_model):
        self.z_model = z_model
        self.n_heights = len(z_model)
        self.all_species = []
        self.load(reactions_file)
        self.time = 0
        self.iteration_step = 0
        self.recording = []

    def getConstituentsIDByName(self, names):
        """
        Gets the ID from a constituents name
        """
        ids = np.array([], dtype = int)
        for name in names:
            for c in self.all_species:
                if c.name == name: ids = np.append(ids, c.c_ID)
        return ids
    
    def load(self, reactions_file):
        """
        Load config file, specifying the species and reactions to be observed
        """
        species_raw = np.array([], dtype = object)
        reactions_raw = np.array([], dtype = str)
        with open(reactions_file, 'r') as f:
            content = f.read()
            lines = [line for line in content.split('\n')]
            for line_no, line in enumerate(lines):
                if line[:14] == '--Constituents':
                    start_consprint = line_no
                if line[:11] == '--Reactions':
                    start_reactions = line_no
                    
            for line_no, line in enumerate(lines):
                if line_no > start_consprint+1 and line == '': break
                if line_no > start_consprint+1:
                    species_raw = np.append(species_raw, line)
        
            for line_no, line in enumerate(lines):
                if line_no > start_reactions+1 and line == '': break
                if line_no > start_reactions+1:
                    reactions_raw = np.append(reactions_raw, line)

        species_str = np.array([c.replace('\t', '').replace(' ', '').split(';') for c in species_raw])
        if con.print: print(species_str, '\n')
        
        
        reactions_str = np.array([r.replace('\t', '').split(';') for r in reactions_raw], dtype = object)
        if con.print: print(reactions_raw)
        if con.print: print(reactions_str)
        
        for c in species_str:
            self.all_species.append(constituent(c[1], c[0], int(c[2]), np.zeros(self.n_heights)))
            
        for ind, c in enumerate(self.all_species):
            if con.print: print(c.name, c.c_ID)
            if c.c_ID != ind: raise RuntimeError
        self.ions = [c for c in self.all_species if c.charge == 1]
        
        self.all_reactions = []
        """
        arrays:
        axis 0: reactions
        axis 1: ID, reaction rate, educts,     products
        axis 2:                    e1, ...     p1, p2, ...
        """
        for r in reactions_str:
            r_ID = r[0]
            r_rate_string = r[2].replace('m3s-1', '').replace(' ', '')
            r_stoch = r[1][1:]
            
            educts, products = r[1].split('=>')
            educts = educts.split(' + ')
            educts = np.char.replace(educts, ' ', '')
        
            products = products.split(' + ')
            products = np.char.replace(products, ' ', '')
            if con.print: print(r_ID, r_rate_string, educts, products)
            
            self.all_reactions.append(reaction(self, r_ID, r_stoch, educts, products, r_rate_string, self.all_species))
    
    
    def assign_densities(self
                         , z_model
                         , z_msis
                         , n_o1_msis
                         , n_n2_msis
                         , n_o2_msis
                         , z_iri
                         , ne_iri        
                         , rel_o_p   
                         , rel_n_p   
                         , rel_h_p   
                         , rel_he_p  
                         , rel_o2_p  
                         , rel_no_p):
        self.all_species[0].density  = np.exp(interp1d(z_iri[1:], np.log(ne_iri[1:]), fill_value = 'extrapolate')(z_model))
        self.all_species[1].density  = np.exp(CubicSpline(z_msis, np.log(n_o1_msis))(z_model))
        self.all_species[2].density  = self.all_species[0].density*interp1d(z_iri, rel_o_p, fill_value='extrapolate')(z_model)
        self.all_species[3].density  = CubicSpline(z_msis, n_o2_msis)(z_model)
        self.all_species[4].density  = self.all_species[0].density*np.exp(interp1d(z_iri, np.log(rel_o2_p), fill_value='extrapolate')(z_model))
        self.all_species[5].density  = self.all_species[0].density*0
        self.all_species[6].density  = self.all_species[0].density*0
        self.all_species[7].density  = CubicSpline(z_msis, n_n2_msis)(z_model)
        self.all_species[8].density  = self.all_species[0].density*0
        self.all_species[9].density  = self.all_species[0].density*0
        self.all_species[10].density = self.all_species[0].density*np.exp(interp1d(z_iri, np.log(rel_no_p), fill_value='extrapolate')(z_model))
        
        self.all_species[0].density  = np.sum([c.density for c in self.all_species if c.charge ==1], axis = 0)
        
        if con.print:
            import matplotlib.pyplot as plt
            plt.figure()
            plt.plot(self.all_species[0].density, z_model, label='e')
            plt.plot(np.sum([i.density for i in self.all_species if i.charge == 1], axis = 0), z_model, label='p')
            plt.legend()
            plt.xscale('log')
        
        
    def update_timestep(self, dt):
        """
        iteration step over timeintervalls.
        """
        for r in self.all_reactions:
            #if con.print: print(r.educts, ' => ', r.products, r.r_rate_string)
            #update Te!!
            Te = 300
            rr = r.r_rate(Te)
            for e_ind in r.educts_ID: #refer to educts by index instead of name
                n = self.all_species[e_ind].density
                rr = rr*n
            for e_ind in r.educts_ID:
                self.all_species[e_ind].loss += rr
            for p_ind in r.products_ID:
                self.all_species[p_ind].prod += rr
                
        for c in self.all_species:
            c.iterate_time(dt)
        
        self.time = self.time + dt
        self.iteration_step += 1
        self.check_chargeNeutrality()
        
    def check_chargeNeutrality(self):
        total_charge = np.zeros(len(self.all_species[0].density))
        density_cc = np.zeros(len(self.all_species[0].density)) 
        for c in self.all_species:
            if c.charge != 0:
                total_charge += c.density * c.charge
                density_cc += c.density
        reduced_tot_charge = total_charge / density_cc
        if any(reduced_tot_charge > 1e-12):
            import matplotlib.pyplot as plt
            plt.figure()
            plt.plot(reduced_tot_charge, self.z_model/1e3)
            plt.xlabel('Charge density [C m-3]')
            plt.ylabel('Altitude [km]')
            plt.title('Charge Density profile at time ' + str(self.time) + ', step: ' + str(self.iteration_step))
            raise RuntimeError('Charge neutrality violated')
        
        if con.print:
            import matplotlib.pyplot as plt
            plt.figure()
            plt.plot(reduced_tot_charge, self.z_model/1e3)
            plt.xlabel('Charge density [C m-3]')
            plt.ylabel('Altitude [km]')
            plt.title('Density profiles')
        


importing Jupyter notebook from organizing.ipynb


In [2]:
"""

if con.print: import matplotlib.pyplot as plt
def assign_densities(z_model
                     , z_msis
                     , n_o1_msis
                     , n_n2_msis
                     , n_o2_msis
                     , z_iri
                     , ne_iri        
                     , rel_o_p   
                     , rel_n_p   
                     , rel_h_p   
                     , rel_he_p  
                     , rel_o2_p  
                     , rel_no_p):
    
    print(all_species[0].name)
    all_species[0].density = np.exp(interp1d(z_iri[1:], np.log(ne_iri[1:]), fill_value = 'extrapolate')(z_model))
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        #plt.plot(, z_msis/1e3, label = 'MSIS data', marker = 'x')
        plt.plot(all_species[0].density, z_model/1e3, label = 'ne_iri (?)')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of e')
        plt.legend()
    
    all_species[1].density = np.exp(CubicSpline(z_msis, np.log(n_o1_msis))(z_model))
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(n_o1_msis, z_msis/1e3, label = 'MSIS data', marker = 'x')
        plt.plot(all_species[1].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of O &' + all_species[1].name)
        plt.legend()
        
    print(all_species[2].name)
    all_species[2].density = all_species[0].density*interp1d(z_iri, rel_o_p, fill_value='extrapolate')(z_model)
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(rel_o_p, z_iri/1e3, label = 'IRI data', marker = 'x')
        plt.plot(all_species[2].density/all_species[0].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of O+')
        plt.legend()
    
    print(all_species[3].name)
    all_species[3].density = CubicSpline(z_msis, n_o2_msis)(z_model)
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(n_o2_msis, z_msis/1e3, label = 'MSIS data', marker = 'x')
        plt.plot(all_species[3].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of O2')
        plt.legend()
        
    print(all_species[4].name)
    #all_species[4].density = all_species[0].density*CubicSpline(z_iri, rel_o2_p)(z_model)
    all_species[4].density = all_species[0].density*np.exp(interp1d(z_iri, np.log(rel_o2_p), fill_value='extrapolate')(z_model))
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(rel_o2_p, z_iri/1e3, label = 'IRI data', marker = 'x')
        plt.plot(all_species[4].density/all_species[0].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of O2+')
        plt.legend()
    
    print(all_species[5].name)
    all_species[5].density = all_species[0].density*0
    
    print(all_species[6].name)
    all_species[6].density = all_species[0].density*0
    
    print(all_species[7].name)
    all_species[7].density = CubicSpline(z_msis, n_n2_msis)(z_model)
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(n_n2_msis, z_msis/1e3, label = 'MSIS data', marker = 'x')
        plt.plot(all_species[7].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of N2')
        plt.legend()
    
    print(all_species[8].name)
    all_species[8].density = all_species[0].density*0
    
    print(all_species[9].name)
    all_species[9].density = all_species[0].density*0
    
    print(all_species[10].name)
    #all_species[10].density = all_species[0].density*CubicSpline(z_iri, rel_no_p)(z_model)
    all_species[10].density = all_species[0].density*np.exp(interp1d(z_iri, np.log(rel_no_p), fill_value='extrapolate')(z_model))
    if con.print:
        plt.figure()
        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')
        plt.plot(rel_no_p, z_iri/1e3, label = 'IRI data', marker = 'x')
        plt.plot(all_species[10].density/all_species[0].density, z_model/1e3, label = 'interp')
        plt.xscale('log')
        plt.xlabel('Density [m-3]')
        plt.ylabel('Altitude [km]')
        plt.title('Density profiles of NO+')
        plt.legend()



"""

"\n\nif con.print: import matplotlib.pyplot as plt\ndef assign_densities(z_model\n                     , z_msis\n                     , n_o1_msis\n                     , n_n2_msis\n                     , n_o2_msis\n                     , z_iri\n                     , ne_iri        \n                     , rel_o_p   \n                     , rel_n_p   \n                     , rel_h_p   \n                     , rel_he_p  \n                     , rel_o2_p  \n                     , rel_no_p):\n    \n    print(all_species[0].name)\n    all_species[0].density = np.exp(interp1d(z_iri[1:], np.log(ne_iri[1:]), fill_value = 'extrapolate')(z_model))\n    if con.print:\n        plt.figure()\n        #plt.plot(alt_dens, z_model/1e3, label = 'interp_old')\n        #plt.plot(, z_msis/1e3, label = 'MSIS data', marker = 'x')\n        plt.plot(all_species[0].density, z_model/1e3, label = 'ne_iri (?)')\n        plt.xscale('log')\n        plt.xlabel('Density [m-3]')\n        plt.ylabel('Altitude [km]')\n  